# **ETL - ATLÁNTICO**

In [1]:
# [Configuración] Importación de librerías
import pandas as pd
import numpy as np
import unicodedata
import re
import os

In [2]:
# [Configuración] Lectura de los archivos CSV
ruta = "../DATA/raw_atlantico" 
archivos = [f for f in os.listdir(ruta) if f.endswith(".csv")]
dataframes = {}
for archivo in archivos:
    path = os.path.join(ruta, archivo)
    df = pd.read_csv(path, encoding='Latin_1', low_memory=False)
    df.columns = df.columns.str.lower()
    dataframes[archivo] = df

print(f"✅ {len(dataframes)} archivos cargados desde {ruta}")

✅ 7 archivos cargados desde ../DATA/raw_atlantico


In [3]:
# [Función] Limpiar nombres de columnas
def limpiar_texto(texto):
    if not isinstance(texto, str):
        return texto
    texto = texto.replace("ã±", "ñ").replace("Ã±", "ñ").replace("ï¿½", "ñ")
    texto = unicodedata.normalize("NFKC", texto)
    texto = re.sub(r'\s+', '_', texto.strip().lower())
    return texto

# Limpiar nombres de columnas de todos los DataFrames
for nombre, df in dataframes.items():
    df.columns = [limpiar_texto(col) for col in df.columns]
    dataframes[nombre] = df

In [4]:
columnas_por_archivo = {
    nombre: set(df.columns)
    for nombre, df in dataframes.items()
}
# Columnas comunes a todos los archivos
comunes = set.intersection(*columnas_por_archivo.values())

# Columnas únicas por archivo
exclusivas = {
    nombre: columnas - comunes
    for nombre, columnas in columnas_por_archivo.items()
}
print("✅ Columnas comunes a todos los archivos:")
print(sorted(comunes))

print("\n❌ Columnas exclusivas por archivo:")
for nombre, cols in exclusivas.items():
    print(f"📄 {nombre}: {sorted(cols)}")

✅ Columnas comunes a todos los archivos:
['ajuste_', 'area', 'año', 'barrio', 'cbmte_', 'cen_pobla_', 'cer_def_', 'cod_ase_', 'cod_pais_o', 'cod_pre', 'cod_sub', 'departamento', 'desenlace', 'direccion', 'edad', 'fec_aju_', 'fec_arc_xl', 'fec_def_', 'fecha_consulta', 'fecha_hospitalizacion', 'fecha_inicio_sintomas', 'fecha_nacimiento', 'fecha_notificacion', 'fm_fuerza', 'fm_grado', 'fm_unidad', 'gp_carcela', 'gp_desmovi', 'gp_desplaz', 'gp_discapa', 'gp_gestan', 'gp_indigen', 'gp_mad_com', 'gp_migrant', 'gp_otros', 'gp_pobicbf', 'gp_psiquia', 'gp_vic_vio', 'hospitalizacion', 'localidad', 'municipio', 'ndep_notif', 'ndep_resi', 'nit_upgd', 'nmun_notif', 'nmun_resi', 'nom_dil_f_', 'nom_upgd', 'nombre_evento', 'num_ide_', 'ocupacion_', 'pertencia_etnica', 'pri_ape_', 'pri_nom_', 'regimen_salud', 'seg_ape_', 'seg_nom_', 'semana', 'sexo', 'tel_dil_f_', 'telefono_', 'tip_ide_', 'tipo_de_caso', 'uni_med_', 'vereda_']

❌ Columnas exclusivas por archivo:
📄 raw_dengue_2017.csv: ['cod_dpto_o', 'c

In [5]:
# Descartar variables poco informativas
delete = [
'ajuste_', 'barrio', 'cbmte_', 'cen_pobla_', 'cer_def_', 
'cod_ase_', 'cod_pais_o', 'cod_pre', 'cod_sub', 
'direccion', 'fec_aju_', 'fec_arc_xl', 'fec_def_', 'fecha_nacimiento',
'fm_fuerza', 'fm_grado', 'fm_unidad', 
'gp_carcela', 'gp_desmovi', 'gp_desplaz', 'gp_discapa', 'gp_gestan', 'gp_indigen', 
'gp_mad_com', 'gp_migrant', 'gp_otros', 'gp_pobicbf', 'gp_psiquia', 'gp_vic_vio', 
'localidad', 'ndep_notif', 'ndep_resi', 'nit_upgd', 'nmun_notif', 'nmun_resi', 'nom_dil_f_', 'nom_upgd', 
'ocupacion_', 'pri_ape_', 'pri_nom_', 'seg_ape_', 'seg_nom_', 'tel_dil_f_', 'telefono_', 'vereda_',
'cod_dpto_o', 'cod_dpto_r', 'cod_eve', 'cod_mun_o', 'cod_mun_r', 'cod_pais_r','eve_historico',
'nombre_nacionalidad', 'num_ide_', 'sem_ges', 'tip_ide_',  'version'
]

for nombre, data in dataframes.items():
    cols_a_borrar = [col for col in delete if col in data.columns]
    if cols_a_borrar:
        dataframes[nombre] = data.drop(columns=cols_a_borrar)
        print(f"🗑️ En '{nombre}' se eliminaron: {cols_a_borrar}")
    else:
        print(f"✅ En '{nombre}' no había columnas para eliminar.")

🗑️ En 'raw_dengue_2017.csv' se eliminaron: ['ajuste_', 'barrio', 'cbmte_', 'cen_pobla_', 'cer_def_', 'cod_ase_', 'cod_pais_o', 'cod_pre', 'cod_sub', 'direccion', 'fec_aju_', 'fec_arc_xl', 'fec_def_', 'fecha_nacimiento', 'fm_fuerza', 'fm_grado', 'fm_unidad', 'gp_carcela', 'gp_desmovi', 'gp_desplaz', 'gp_discapa', 'gp_gestan', 'gp_indigen', 'gp_mad_com', 'gp_migrant', 'gp_otros', 'gp_pobicbf', 'gp_psiquia', 'gp_vic_vio', 'localidad', 'ndep_notif', 'ndep_resi', 'nit_upgd', 'nmun_notif', 'nmun_resi', 'nom_dil_f_', 'nom_upgd', 'ocupacion_', 'pri_ape_', 'pri_nom_', 'seg_ape_', 'seg_nom_', 'tel_dil_f_', 'telefono_', 'vereda_', 'cod_dpto_o', 'cod_dpto_r', 'cod_eve', 'cod_mun_o', 'cod_mun_r', 'eve_historico', 'num_ide_', 'tip_ide_']
🗑️ En 'raw_dengue_2018.csv' se eliminaron: ['ajuste_', 'barrio', 'cbmte_', 'cen_pobla_', 'cer_def_', 'cod_ase_', 'cod_pais_o', 'cod_pre', 'cod_sub', 'direccion', 'fec_aju_', 'fec_arc_xl', 'fec_def_', 'fecha_nacimiento', 'fm_fuerza', 'fm_grado', 'fm_unidad', 'gp_carc

## Unificación

In [6]:
# Concatenar archivos en un solo DataFrame
data = pd.concat(dataframes.values(), ignore_index=True)
data = data[sorted(data.columns)]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36714 entries, 0 to 36713
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   area                   36714 non-null  int64 
 1   año                    36714 non-null  int64 
 2   departamento           36714 non-null  object
 3   desenlace              36714 non-null  int64 
 4   edad                   36714 non-null  int64 
 5   fecha_consulta         36714 non-null  object
 6   fecha_hospitalizacion  36714 non-null  object
 7   fecha_inicio_sintomas  36714 non-null  object
 8   fecha_notificacion     36714 non-null  object
 9   hospitalizacion        36714 non-null  int64 
 10  municipio              36714 non-null  object
 11  nombre_evento          36714 non-null  object
 12  pertencia_etnica       36714 non-null  int64 
 13  regimen_salud          36714 non-null  object
 14  semana                 36714 non-null  int64 
 15  sexo               

_________

## Agregación

In [7]:
# [Variable] Evento
data.loc[:, 'nombre_evento'] = data['nombre_evento'].str.strip()
data = data[data['nombre_evento'] != 'MORTALIDAD POR DENGUE']
data['evento'] = np.where(data['nombre_evento'] == 'DENGUE', 'Clasico', 
                       np.where(data['nombre_evento'] == 'DENGUE GRAVE', 'Grave', 'Otros'))
data['evento'].unique()

array(['Clasico', 'Grave'], dtype=object)

In [8]:
# [Variable] Edad en años
data["edad_años"] = np.where(data["uni_med_"] == 1, data["edad"],   # ya está en años
                    np.where(data["uni_med_"] == 2, data["edad"] / 12,  # meses → años
                    np.where(data["uni_med_"] == 3, data["edad"] / 365, # días → años
                    np.where(data["uni_med_"] == 4, data["edad"] / (365*24), # horas
                    np.where(data["uni_med_"] == 5, data["edad"] / (365*24*60), # minutos
                    np.where(data["uni_med_"] == 0, np.nan, 
                     np.nan))))))  # por si hay otros valores
data["edad_años"] = data["edad_años"].round().fillna(-1).astype(int)
data['edad_años'].describe()

count    36707.000000
mean        16.703953
std         13.748010
min          0.000000
25%          8.000000
50%         13.000000
75%         20.000000
max        101.000000
Name: edad_años, dtype: float64

In [9]:
# [Variable] Ciclo vital a partir de edad en años
def ciclo_vital(edad):
    if edad < 0:
        return "Desconocido"
    elif edad < 1:
        return "Menor de 1 año"
    elif edad < 5:
        return "Primera infancia"
    elif edad < 10:
        return "Infancia"
    elif edad < 20:
        return "Adolescencia"
    elif edad < 60:
        return "Adulto"
    elif edad < 85:
        return "Adulto mayor"
    else:
        return "Ancianidad"
    
data["ciclo_vital"] = data["edad_años"].apply(ciclo_vital)
data["ciclo_vital"].unique()

array(['Ancianidad', 'Adolescencia', 'Adulto mayor', 'Infancia', 'Adulto',
       'Primera infancia', 'Menor de 1 año'], dtype=object)

In [10]:
# [Mapeo] Variables categóricas
mapeo = {
    'area' : {1: 'Cabecera municipal' , 2:'Centro Poblado', 3:'Rural disperso'}, 
    'hospitalizacion': {1: 'Si', 2: 'No'},
    'desenlace': {1: 'Vivo', 2: 'Fallecido'},
    'pertencia_etnica': {1: 'Indigena', 2: 'Rom', 3: 'Raizal', 4: 'Palenquero', 5: 'Negro', 6:'Otro'},
    'regimen_salud': {'P': 'Excepción', 'E': 'Especial', 'C': 'Contributivo', 'S': 'Subsidiado', 'N': 'No asegurado', 'I': 'Indeterminado'},
    'sexo': {'M': 'Masculino', 'F': 'Femenino'},
    'tipo_de_caso': {2: 'Probable', 3: 'Conf. por laboratorio', 5: 'Conf. por nexo'}
}
data = data.replace(mapeo)

In [11]:
# [Mapeo] Grupos étnicos
mapeo_etnico = {
    'Indigena': 'Indígena',
    'Negro': 'Afrocolombiano',
    'Raizal': 'Afrocolombiano',
    'Palenquero': 'Afrocolombiano',
    'Rom': 'Rom',
    'Otro': 'Otro'
}
data['grupo_etnico'] = data['pertencia_etnica'].map(mapeo_etnico)

In [12]:
# [Variable] Departamento
data['departamento'] = data["departamento"].str.upper().str.strip()
data['departamento'] = data['departamento'].str.replace('_', ' ', regex=False)

data['departamento'] = data['departamento'].replace({'ATLANTICO': 'ATLÁNTICO', '8': 'ATLÁNTICO'})
data = data[data['departamento'] == "ATLÁNTICO"]
data['departamento'].unique()

array(['ATLÁNTICO'], dtype=object)

In [13]:
# [Variable] Departamento
data['municipio'] = data["municipio"].str.upper().str.strip()
data['municipio'] = data['municipio'].str.replace('_', ' ', regex=False)

correcciones_mun= {
    '1': 'BARRANQUILLA',
    '758': 'SOLEDAD',
    '606': 'REPELÓN',
    '372': 'JUAN DE ACOSTA',
    '849': 'USIACURÍ', 'USIACURI': 'USIACURÍ', '8499': 'USIACURÍ',
    '296': 'GALAPA',
    '606': 'REPELÓN', 'REPELON': 'REPELÓN', 
    '78': 'BARANOA', 
    '558': 'POLONUEVO', 'POLO NUEVO': 'POLONUEVO',
    '638': 'SABANALARGA',
    '433': 'MALAMBO',
    '421': 'LURUACO',
    '634': 'SABANAGRANDE',
    '685': 'SANTO TOMÁS', 'SANTO TOMAS': 'SANTO TOMÁS', 
    '832': 'TUBARÁ', 'TUBARA': 'TUBARÁ',
    '137': 'CAMPO DE LA CRUZ',
    '* ATLANTICO. MUNICIPIO DESCONO': 'DESCONOCIDO',
    '560': 'PONEDERA',
    '573': 'PUERTO COLOMBIA', 
    '436': 'MANATÍ', 'MANATI': 'MANATÍ',
    '141': 'CANDELARIA', 
    '560': 'PONEDERA', '560PO': 'PONEDERA',
    '520': 'PALMAR DE VARELA',
    '770': 'SUAN',
    '549': 'PIOJÓ', 'PIOJO': 'PIOJÓ',
    '675': 'SANTA LUCÍA', 'SANTA LUCIA': 'SANTA LUCÍA',
    '0': 'DESCONOCIDO', 
    '7': 'DESCONOCIDO'

}

data['municipio'] = data['municipio'].replace(correcciones_mun)
data['municipio'].unique()

array(['BARRANQUILLA', 'SOLEDAD', 'REPELÓN', 'JUAN DE ACOSTA', 'GALAPA',
       'USIACURÍ', 'BARANOA', 'POLONUEVO', 'SABANALARGA', 'SABANAGRANDE',
       'MALAMBO', 'LURUACO', 'CAMPO DE LA CRUZ', 'SANTO TOMÁS',
       'DESCONOCIDO', 'TUBARÁ', 'PONEDERA', 'PUERTO COLOMBIA', 'MANATÍ',
       'CANDELARIA', 'PALMAR DE VARELA', 'SUAN', 'PIOJÓ', 'SANTA LUCÍA'],
      dtype=object)

In [14]:
def obtener_subregion(mun):
    if mun in ['BARRANQUILLA', 'GALAPA', 'MALAMBO', 'SOLEDAD', 'PUERTO COLOMBIA']:
        return 'Metropolitana'
    elif mun in ['PALMAR DE VARELA', 'PONEDERA', 'SANTO TOMÁS', 'SABANAGRANDE']:
        return 'Oriental'
    elif mun in ['CAMPO DE LA CRUZ', 'CANDELARIA', 'SANTA LUCÍA', 'SUAN', 'REPELÓN', 'MANATÍ']:
        return 'Sur'
    elif mun in ['BARANOA', 'POLONUEVO', 'LURUACO', 'USIACURÍ', 'SABANALARGA']:
        return 'Centro'
    elif mun in ['JUAN DE ACOSTA', 'PIOJÓ', 'TUBARÁ']:
        return 'Costera'
    else:
        return 'Desconocida'

# Aplicamos la función
data['subregion'] = data['municipio'].apply(obtener_subregion)

In [15]:
# [Variable] Periodo epidemiológico (4 semanas)
def asignar_periodo(semana):
    """
    Calcula el número de periodo según la semana epidemiológica.
    Cada periodo abarca 4 semanas. 
    La semana 53 se incluye en el último periodo.
    """
    if pd.isna(semana):
        return np.nan
    semana = int(semana)
    periodo = int(np.ceil(semana / 4))
    if semana == 53:  # Semana extra del año
        periodo = int(np.ceil(52 / 4))  # igual al último periodo
    return periodo

# Aplicar al dataframe
data["periodo"] = data["semana"].apply(asignar_periodo)

In [16]:
data.columns

Index(['area', 'año', 'departamento', 'desenlace', 'edad', 'fecha_consulta',
       'fecha_hospitalizacion', 'fecha_inicio_sintomas', 'fecha_notificacion',
       'hospitalizacion', 'municipio', 'nombre_evento', 'pertencia_etnica',
       'regimen_salud', 'semana', 'sexo', 'tipo_de_caso', 'uni_med_', 'evento',
       'edad_años', 'ciclo_vital', 'grupo_etnico', 'subregion', 'periodo'],
      dtype='object')

In [17]:
# Descartar variables originales ya transformadas
data = data.drop(columns=['edad', 'uni_med_', 'pertencia_etnica', 'nombre_evento', 'departamento'])

In [18]:
# [Data] Estructuración por temática

    # Variables demográficas
demograficas = [
    "sexo",                    # M/F
    "edad_años",               # Edad numérica
    "ciclo_vital",             # Etapa de vida
    "grupo_etnico",            # Grupo étnico declarado
    "regimen_salud"            # Régimen (contributivo, subsidiado, etc.)
]

    # Variables del evento epidemiológico
evento = [
    "evento",                  # Clásico, grave
    "tipo_de_caso",               # Confirmado, probable, descartado
    "hospitalizacion",         # Sí/No
    "desenlace"                # Recuperado, fallecido, etc.
]

    # Variables de temporalidad
temporalidad = [
    "año",                     # Año del evento
    "semana",                  # Semana epidemiológica
    "periodo",                 # Periodo (cada 4 semanas)
    "fecha_inicio_sintomas",   # Inicio de síntomas
    "fecha_consulta",          # Fecha de consulta
    "fecha_hospitalizacion",   # Fecha de hospitalización
    "fecha_notificacion",      # Fecha de notificación
]

    # Variables espaciales
espacial = [
    "subregion",                # Subregion del Atlántico
    "municipio",                # Municipio de ocurrencia
    "area",                     # Urbana o rural
]

orden_columnas = demograficas + evento + temporalidad + espacial
cols_existentes = [col for col in orden_columnas if col in data.columns]
data = data[cols_existentes + [col for col in data.columns if col not in cols_existentes]]

In [19]:
# DataFrame final
data.shape

(35845, 19)

In [20]:
df_atl = data[data['municipio'] != "BARRANQUILLA"]
df_atl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18578 entries, 7 to 36713
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   sexo                   18578 non-null  object
 1   edad_años              18578 non-null  int64 
 2   ciclo_vital            18578 non-null  object
 3   grupo_etnico           18578 non-null  object
 4   regimen_salud          18578 non-null  object
 5   evento                 18578 non-null  object
 6   tipo_de_caso           18578 non-null  object
 7   hospitalizacion        18578 non-null  object
 8   desenlace              18578 non-null  object
 9   año                    18578 non-null  int64 
 10  semana                 18578 non-null  int64 
 11  periodo                18578 non-null  int64 
 12  fecha_inicio_sintomas  18578 non-null  object
 13  fecha_consulta         18578 non-null  object
 14  fecha_hospitalizacion  18578 non-null  object
 15  fecha_notificacion     1

In [21]:
df_atl.to_csv(r"C:\Users\Hp\DENGUE\DATA\Data_Atlantico.csv", index=False, encoding='utf-8')